In [1]:
from room import Room
from person import Person
from action import Action
import metacode_helpers

In [2]:
import pandas as pd
import numpy as np

In [3]:
# initialise all actions
ALL_POSSIBLE_ACTIONS = metacode_helpers.initialise_all_actions()
ALL_CONVERSATION_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "conversation"]
ALL_ROOM_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "room"] 

In [4]:
# Create some rooms
HALL = Room("Hall")
KITCHEN = Room("Kitchen")
LOUNGE = Room("Lounge")

# Add adjacencies to the rooms (HALL is adjacent to KITCHEN and LOUNGE)
HALL.add_adjacent_room(KITCHEN)
HALL.add_adjacent_room(LOUNGE)

# Assign the Hall to be the entry to the house
HOUSE_ENTRY_ROOM = HALL 

ROOMS = [HALL, KITCHEN, LOUNGE]

In [5]:
names = metacode_helpers.get_names(num_names=32)
np.random.seed(42)
ALL_PEOPLE = []
for personality_number in range(4):
    personality_vector = np.random.uniform(low = -1, high = 1, size = 5)
    initial_emotional_state_vector = np.random.uniform(low = -1, high = 1, size = 3)
    initial_location_state = np.random.choice(ROOMS)
    new_person = Person(name = names[personality_number], location_state=initial_location_state, all_possible_actions=ALL_POSSIBLE_ACTIONS, personality_vector=personality_vector, emotional_state_vector=initial_emotional_state_vector, conversation_partner=None)
    ALL_PEOPLE.append(new_person)

In [6]:
ALL_PEOPLE

[Robin, Patrick, Ronnie, Jacquelyn]

In [7]:
ALL_POSSIBLE_ACTIONS

array([shouts, shares, leaves_conversation, makes_smalltalk,
       makes_deeptalk, gives_controversial_opinion, changes_the_topic,
       shows_interest, shows_concern, laughs, jokes, gives_an_insult,
       brags, yawns, supports, rants, stands_in_corner, leaves_room,
       starts_conversation], dtype=object)

In [11]:
from collections import defaultdict

action_log = defaultdict(list)
person_before_action_log = defaultdict(list)
person_after_action_log = defaultdict(list)
partner_before_action_log = defaultdict(list)
partner_after_action_log = defaultdict(list)
room_log = defaultdict(list)

def log_snapshot(log, snapshot, turn_number, round_number):
    log["turn_number"].append(turn_number)
    log["round_number"].append(round_number)
    for key, val in snapshot.items():
        log[key].append(val)

turn_number = 0
for round_number in range(10):
    for person in ALL_PEOPLE:
        partner = person.get_conversation_partner()
        log_snapshot(person_before_action_log, person.get_snapshot(), turn_number, round_number)
        if person.has_conversation_partner():
            log_snapshot(partner_before_action_log, person.get_conversation_partner().get_snapshot(), turn_number, round_number)

        # The person can start a conversation only if there's someone free to chat in their current room.
        available_room_act = ALL_ROOM_ACTIONS if person.get_location_state().is_someone_free_to_chat() else [action for action in ALL_ROOM_ACTIONS if action.get_name() != "starts_conversation"]

        # If in a conversation, the person can select any action, including leaving.
        action = person.action_selection(available_conv_act=ALL_CONVERSATION_ACTIONS) if person.has_conversation_partner() else person.action_selection(available_room_act=available_room_act)
        
        # If in a conversation, update the person & conversation partner's emotional state according to the action
        if person.has_conversation_partner():
            # Change this person's emotional state 
            person.update_emotional_state_vector(action, isOwnAction = True)
            # Change the partner's emotional state 
            partner.update_emotional_state_vector(action, isOwnAction = False)

            if action.get_name() == "leaves_conversation":
                # The conversation has ended, so both people now don't have conversation partners.
                person.set_conversation_partner(None)
                partner.set_conversation_partner(None)
                
        else: # Otherwise, deal with the room action
            if action.get_name() == "leaves_room":
                next_room = person.get_next_room()
                person.move_to_room(next_room)          
            if action.get_name() == "starts_conversation":
                # NOTE whether this is possible is handled by the action selection function
                # Find a person in the room who is free to chat
                current_room = person.get_location_state()
                partner = current_room.get_new_conversation_partner_for(person)
                person.set_conversation_partner(partner)
                partner.set_conversation_partner(person)
        
        log_snapshot(action_log, action.get_snapshot(), turn_number, round_number)    
        log_snapshot(person_after_action_log, person.get_snapshot(), turn_number, round_number)
        if partner:
            log_snapshot(partner_after_action_log, partner.get_snapshot(), turn_number, round_number)

        message = person.get_name(), ' '.join(action.get_name().split("_"))
        print(round_number)
        print(turn_number)
        print(message)
        turn_number += 1

0
0
('Robin', 'stands in corner')
0
1
('Patrick', 'makes smalltalk')
0
2
('Ronnie', 'shouts')
0
3
('Jacquelyn', 'leaves room')
1
4
('Robin', 'leaves room')
1
5
('Patrick', 'shows concern')
1
6
('Ronnie', 'laughs')


ValueError: 'a' cannot be empty unless no samples are taken

In [9]:
pd.DataFrame(action_log)

,turn_number,round_number,name
0,0,0,leaves_room
1,1,0,stands_in_corner
2,2,0,leaves_room
3,3,0,leaves_room
4,4,1,stands_in_corner
5,5,1,leaves_room
6,6,1,stands_in_corner
7,7,1,leaves_room
8,8,2,starts_conversation
9,9,2,leaves_room
